# 次元圧縮

次元圧縮
- 文章をトピックの集まりとすることで特徴量を削減

潜在的意味解析 (LSA) 
- 同じように使われる単語をトピックとしてまとめる方法

<hr>

![LSA](slides/LSA.png)

- 行列 U：各文書におけるトピックの擬似的な頻度
- 行列 S：重要度（対角成分）
- 行列 V：各トピックに単語が現れる度合い
<hr>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer

# matplotlib: 日本語フォントの設定
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 
                               'Takao', 'IPAexGothic', 'IPAPGothic', 'Noto Sans CJK JP']

# 日本語モデル
nlp = spacy.load('ja_core_news_lg')

# フィードデータの読み込み、確認
feeds = pd.read_csv('data/output_jp.csv')

# title と summary を結合
# str.cat() により複数列の文字列を結合
# - sep=' ': 間に挟む文字列
# - na_rep='': NaN は空文字列に変換（指定しないと結合結果が NaN になる）
feeds['text'] = feeds['title'].str.cat(feeds['summary'], sep='。', na_rep='')

# 不要になった列を削除した処理用の DataFrame
df = feeds.drop(['title', 'summary'], axis=1)

# 確認
df

### 日本語テキストに対する前処理

- 表記の正規化
- トークン化（形態素解析）
- ストップワードの除去
- 見出し語化

In [ ]:
# 不要な単語を除去
# - ストップワード (is_stop)
# - いくつかの品詞
#     AUX: 助動詞
#     PUNCT: 句読点
#     SPACE: 空白文字
#     SYM: 記号
#     X: その他
# - うまく取り除けない単語や文字
stop_pos = ['AUX', 'PUNCT', 'SPACE', 'SYM', 'X']
stop_words = ['.']

def token_to_add(w):
    t = w.text    # 単語
    p = w.pos_    # 品詞
    l = w.lemma_  # 原型

    # ストップワードは None を返す
    if w.is_stop:
        return None
    if p in stop_pos:
        return None
    if l in stop_words:
        return None

    if len(l) == 0:
        return t
    return l

def preprocess(text):
    tokens = []
    
    for w in nlp(text):
        t = token_to_add(w)
        if t is not None:
            tokens.append(t)

    # トークンのリストを返す
    return tokens

### テキストのベクトル化

- Bag of Words (BoW)

In [ ]:
# 初期化
vectorizer = CountVectorizer(tokenizer=preprocess)
# ベクトル化
vector = vectorizer.fit_transform(df.text)
# テキストをベクトル化した結果の DataFrame
df_vector = pd.DataFrame.sparse.from_spmatrix(vector)
# 確認
df_vector

In [ ]:
# 単語の DataFrame
df_words = pd.DataFrame(vectorizer.get_feature_names_out())
# 列インデックス
df_words.columns = ['word']
# 確認
df_words

テキスト数に対し単語数の方が（圧倒的に）多いと、アルゴリズムによってはうまく動かなくなる場合が出てくる。

## LSA: 潜在的意味解析

In [ ]:
from sklearn.decomposition import TruncatedSVD

# 初期化
# - n_components=100: トピック数（100はドキュメントでの推奨値）
model = TruncatedSVD(n_components=100)

# トピックへの分解
# - スライドの行列 U
lsa_features = model.fit_transform(vector)

# トピックによる説明割合
# - 割合が大きくなるようにトピック数を選択することも行われる
print('explained_variance_ratio: {:.3f}'.format(model.explained_variance_ratio_.sum()))

# サイズを確認
print('shape:', lsa_features.shape)

# pandas DataFrame 化
df_lsa = pd.DataFrame(lsa_features)

In [ ]:
# 単語とトピックを対応させたDataFrameを作成
# - model.components_: 各トピックにおける単語との対応（スライドの行列 V）
df_components = pd.concat([df_words, pd.DataFrame(model.components_).T], axis=1)
df_components = df_components.set_index('word')
df_components

In [ ]:
# 最初の10個をトピックの重要度と合わせて表示
# - 重要度: スライドの行列 S の対角成分
# - 単語: 各トピックに単語が現れる度合い
# df_components.sort_values(i, ascending=False)
# - i 列を降順でソート
for i in range(0, 10):
    print('[{}] {:.2f} {}'.format(i, model.singular_values_[i],
                                  df_components.sort_values(i, ascending=False).head(10).index.values))

### 非階層的クラスタリング

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

# クラスタ数
N_clusters = 10

# KMeansによりクラスタリング
# - lsa_features: スライドの行列 U 各文書におけるトピックの擬似的な頻度
# - normalize() により単位ベクトル化
clusters = KMeans(n_clusters=N_clusters).fit_predict(normalize(lsa_features))

# 結果を DataFrame にまとめる
df_cluster = pd.DataFrame(clusters, columns=['cluster'])

### クラスタごとの棒グラフ（頻度）

In [ ]:
# 頻度の棒グラフ
for i in range(0, N_clusters):
    print('Cluster', i)
    _cdf = df_cluster[df_cluster['cluster'] == i]
    df_counts = pd.concat([df_words, df_vector.iloc[_cdf.index].sum()], axis=1)
    df_counts.columns=['word', 'counts']
    df_bar = df_counts.sort_values('counts', ascending=False).head(10)
    sns.barplot(x=df_bar.counts, y=df_bar.word, orient='h')
    # 各クラスタごとの主要 3 トピックから上位 10 単語を表示
    for j in df_lsa.iloc[_cdf.index].mean().sort_values(ascending=False).head(3).index:
        print('[{}] {}'.format(j, df_components.sort_values(j, ascending=False).head(10).index.values))
    # 棒グラフの表示
    plt.show()

### 分類

In [ ]:
label_0 = ['business', 'economy', 'politics', 'cat4', 'cat5']
label_1 = ['culture', 'science', 'sport', 'cat2', 'cat7']

# label_0 の単語が url の中に含まれている：0
# label_1 の単語が url の中に含まれている：1
# それ以外：2
def get_label(url):
    url = url.lower()
    # map: label_0 の単語それぞれについて、url の中に含まれているか真偽を返す
    # any: mapの結果について論理和をとる
    if any(map(lambda x: x in url, label_0)):
        return 0
    if any(map(lambda x: x in url, label_1)):
        return 1
    else:
        return 2

# df['url'] について get_label を適用した結果を df['label'] として追加
df['label'] = df['url'].map(lambda x: get_label(x))

# label 0, 1 を取り出す
df = df.query('label != 2')

# 初期化
vectorizer = CountVectorizer(tokenizer=preprocess)

# ベクトル化
vector = vectorizer.fit_transform(df.text)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

# 説明変数、目的変数
# - vector が sparse のため toarray() により dense に変換
X = vector.toarray()
Y = df.label

# ナイーブベイズ分類器
model = MultinomialNB()

# 交差検証の実行
score = cross_val_score(model, X, Y, scoring='roc_auc')
print('AUC={:.3f} (+/- {:.3f})'.format(score.mean(), score.std()))

### トピックの分類

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 初期化
# - n_components=100: トピック数（100はドキュメントでの推奨値）
LSA = TruncatedSVD(n_components=100)

# トピックへの分解
lsa_features = LSA.fit_transform(vector)

# lsa_features には負の値も出てくるため [0, 1] に正規化
scaler = MinMaxScaler()
X = scaler.fit_transform(lsa_features)

# ナイーブベイズ分類器
model = MultinomialNB()

# 交差検証の実行
score = cross_val_score(model, X, Y, scoring='roc_auc')
print('AUC={:.3f} (+/- {:.3f})'.format(score.mean(), score.std()))